In [1]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models.ollama import ChatOllama
from langchain_objectbox.vectorstores import ObjectBox
from langchain.prompts import ChatPromptTemplate
from objectbox import Store
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS

import os
import shutil

In [2]:
EMBEDDING = OllamaEmbeddings(base_url="http://192.168.1.101:11434", model="mxbai-embed-large")
CHAT_MODEL = ChatOllama(base_url="http://192.168.1.101:11434", model='llama3.1', embedding=EMBEDDING)

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [3]:
objectbox = ObjectBox(embedding=EMBEDDING, embedding_dimensions=1024)

In [5]:
def vector_search_db(query_text):
    query_embedding = EMBEDDING.embed_query(query_text)
    db_results = objectbox.similarity_search_by_vector(query_embedding, k=6)
    return db_results

def generate_prompt(db_results, query_text):
    context_text = "\n\n---\n\n".join([doc.page_content for doc in db_results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    return prompt

def generate_response(prompt):
    return CHAT_MODEL.invoke(prompt)

def format_response(response_text, db_results):
    sources = set([doc.metadata.get("source", None) for doc in db_results])
    sources = [source.split("/",1)[-1].rsplit(".",1)[0] for source in sources if source is not None]
    return f"Response: {response_text}\nSources: {sources}"

In [6]:
retriever = objectbox.as_retriever()
query = "What are the duties of the secretary of state?"
db_results = retriever.invoke(query, k=6)
print(len(db_results), "results found.")
prompt = generate_prompt(db_results, query)
response = generate_response(prompt)
formatted_response = format_response(response, db_results)
print(formatted_response)

4 results found.


/workspaces/MigraChat/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response: According to the provided context, the Secretary of State's duties include:

1. Administration and enforcement of provisions in this chapter and all other immigration and nationality laws.
2. Powers and duties concerning United States citizens, including those related to passports.
3. Amending or revoking part 46 of title 22, Code of Federal Regulations, which concerns persons other than US citizens.

Additionally, the Secretary of State is charged with the administration and enforcement of provisions related to:

1. Diplomatic and consular officers' powers, duties, and functions (except for those related to visa granting or refusal).
2. The Administrator's powers, duties, and functions.
3. Determination of a person's nationality.

Overall, the Secretary of State has broad authority and responsibility for various aspects of immigration, nationality, and diplomatic matters within the Department of State.
Sources: ['8 USC 1185 Travel control of citizens and aliens', '8 USC 1104

In [7]:
query = "What are the duties of the secretary of state?"
db_results = vector_search_db(query)
if db_results:
    prompt = generate_prompt(db_results, query)
    response = generate_response(prompt)
    print(format_response(response, db_results))

Response: Based on the provided context, the duties of the Secretary of State include:

1. Administering and enforcing provisions of this chapter (Immigration and Nationality Act) and all other immigration and nationality laws.
2. Exercising powers and functions under section 215(a) and (b) of the INA with respect to citizens of the United States, including those concerning United States passports.
3. Amending or revoking part 46 of title 22, Code of Federal Regulations, which concerns persons other than citizens of the United States.
4. Maintaining close liaison with the appropriate committees of Congress in order that they may be advised regarding the administration of this chapter by consular officers.

The Secretary of State is also responsible for designating an Administrator who shall maintain close liaison with the appropriate committees of Congress and perform duties as designated by the Secretary of State.
Sources: ['8 USC 1185 Travel control of citizens and aliens', '8 USC 11

In [8]:
CHAT_MODEL = ChatOllama(base_url="http://192.168.1.101:11434", model='tinyllama', embedding=EMBEDDING)

In [9]:
retriever = objectbox.as_retriever()
query = "What are the duties of the secretary of state?"
db_results = retriever.invoke(query, k=6)
print(len(db_results), "results found.")
prompt = generate_prompt(db_results, query)
response = generate_response(prompt)
formatted_response = format_response(response, db_results)
print(formatted_response)

4 results found.
Response: The power and duties of the Secretary of State are:
(a) Powers and duties

-- (b) Functions of the Secretary of State

To:
1. Amend or revoke part 46 of Title 22, Code of Federal Regulations, concerning persons other than citizens of the United States

To:
2. Administer and enforce immigration and nationality laws relating to (1) powers, duties, and functions of diplomatic and consular officers of the United States, except those powers, duties, and functions conferred upon the Consular Officers relating to the granting or refusal of visas;
(2) Powers, duties, and functions of the Administrator; and
(3) Determination of nationality of a person.
Sources: ['8 USC 1185 Travel control of citizens and aliens', '8 USC 1104 Powers and duties of Secretary of State']


In [19]:
query = "What are the duties of the secretary of state?"
db_results = vector_search_db(query)
if db_results:
    prompt = generate_prompt(db_results, query)
    response = generate_response(prompt)
    print(format_response(response, db_results))

Response: The section states that the Secretary of State is assigned the functions of the President under Section 215(a) and (b) of the INA with respect to citizens of the United States, including those functions concerning United States passports. In exercising these functions, the Secretary of State shall be charged with the administration and the enforcement of immigration and nationality laws relating to persons other than citizens of the United States. The section further states that the Secretary of State has power to have and exercise any authority vested by law in any official or office of Department of State and references to such officials or offices deemed to refer to the Secretary of State, or the Department of State, as appropriate. Therefore, the section specifies the duties of the secretary of state as those related to United States passports and nationality laws.
Sources: ['8 USC 1185 Travel control of citizens and aliens', '8 USC 1104 Powers and duties of Secretary of 

In [16]:
import requests

# make post request to 192.168.1.101/query

query = "What are the duties of the secretary of state?"
response = requests.post("http://192.168.1.101:8000/query", json={"query": query})
print(response.text)

{"response":"Response: content='The Secretary of State is assigned the functions of the President under section 215(a) and (b) of the INA with respect to citizens of the United States, including those functions concerning United States passports. In addition, the Secretary may amend or revoke part 46 of title 22, Code of Federal Regulations, which concern persons other than citizens of the United States. The Secretary shall have and exercise any authority vested by law in any official or office of Department of State, and shall maintain close liason with the approprite committees of Congress to ensure that they can be advised regarding the administration of this chapter by consular officers. The Administrator is a citizen of the United States qualified by experience who maintains close liasion with the approprite committees of Congress, and shall perform such other duties as are designated by the Secretary of State.' response_metadata={'model': 'tinyllama', 'created_at': '2024-08-02T13